In [1]:
import statsmodels.api as sm
import numpy as np
from scipy import stats
from matplotlib import pyplot as plt

In [6]:
A = np.array([[83, 85], [84, 85, 85, 86, 86, 87], \
     [86, 87, 87, 87, 88, 88, 88, 88, 88, 89, 90], \
     [89, 90, 90, 91], [90, 92]])
N = 25

<ipython-input-6-8b3e9f5fa655>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  A = np.array([[83, 85], [84, 85, 85, 86, 86, 87], \


In [7]:
def rss0(x):
    return sum([(a - np.mean(x))**2 for a in x])

def regression(psi, eta):
    F = psi.T @ psi
    F_inv = np.linalg.inv(F)
    b = (F_inv @ psi.T) @ eta.T
    e = eta - (psi @ b.T).T
    RSSy = e @ e.T
    RSS0y = rss0(eta)

    R = (RSS0y - RSSy)/RSS0y
    
    delta = [b[i]*((N - len(psi[0]))/(RSSy*F_inv[i][i]))**(1/2) \
             for i in range(len(psi[0]))]
    b_p_value = 2*(1 - stats.t.cdf(np.abs(delta), N - np.shape(psi)[1]))
    
    return b, b_p_value, RSSy, RSS0y, R

## a)

In [8]:
k = 0
size = 0
for a in A:
    size += len(a)

psi = np.zeros((size, len(A)))
eta = np.zeros(size)

for i in range(len(A)):
    for j in range(len(A[i])):
        eta[k] = A[i][j]
        psi[k][i] = 1
        k += 1

F = psi.T @ psi
F_inv = np.linalg.inv(F)
    
b, b_p_value, RSS, RSS0, R = regression(psi, eta)

print("b =", np.around((b), decimals=3))
print("p-value =", np.around((b_p_value), decimals=3))
print()
print ("R =", "%.3f" % R)

delta_R = R/(1 - R)*(size - len(A))/(len(A) - 1)
R_p_value = 1 - stats.f.cdf(np.abs(delta_R), \
                            len(A) - 1, size - len(A))
print("p-value =", "%.3f" % R_p_value)

b = [84.    85.5   87.818 90.    91.   ]
p-value = [0. 0. 0. 0. 0.]

R = 0.811
p-value = 0.000


## b)

In [9]:
delta_ij = np.zeros(int(len(A)*(len(A) - 1)/2))
p_value_ij = {}

for i in range(len(A)):
    for j in range(i + 1, len(A)):
        l = i*(len(A)-1-i) + j-1 + (i-1)*((i-1) > 0) + ((i-2) > 0)
        
        delta_ij[l] = np.abs((b[i] - b[j])*\
                                (size-len(A))**(1/2)/(RSS*\
                                 (F_inv[i][i] + F_inv[j][j]))**0.5)
        p_value_ij[str(i)+'<->'+str(j)] = np.around((2*\
            (1 - stats.t.cdf(delta_ij[l], size-len(A)))), decimals=3)

print(" p-value(ij): ", p_value_ij)

 p-value(ij):  {'0<->1': 0.103, '0<->2': 0.0, '0<->3': 0.0, '0<->4': 0.0, '1<->2': 0.0, '1<->3': 0.0, '1<->4': 0.0, '2<->3': 0.002, '2<->4': 0.001, '3<->4': 0.296}
 возрастные группы 0,1 и 3,4 похожи


из p-value > 0.05 делаем вывод, что возростные группы:
0 и 1,
3 и 4 - похожи